In [82]:
import pandas as pd
from os import path
import os

DATA_IN = "../data/raw/semi-raw_data.csv"
assert path.exists(DATA_IN)

df = pd.read_csv(DATA_IN, index_col=0)

In [83]:
df.columns

Index(['OBJECTID', 'UNITID', 'SIGID', 'MAINSTREET', 'CROSSSTREE', 'OWNER2',
       'ROUTE', 'MILEPOINT', 'DESCRIPTIO', 'TIMES', 'OWNER', 'TYPE', 'INTID',
       'ATMSID', 'geometry'],
      dtype='object')

In [84]:
renames = {
    'CROSSSTREE': 'cross_street',
    'DESCRIPTIO': 'desription',
    'INTID': 'int_ID',
    'MAINSTREET': 'main_street',
    'MILEPOINT': 'milepoint',
    'OBJECTID': 'row_ID',
    'OWNER': 'owner',
    'ROUTE': 'route',
    'SIGID': 'sig_ID',
    'TIMES': 'times',
    'TYPE': 'type',
    'UNITID': 'unit_ID'
}

drops = ['OWNER2']

def clean_columns(df):
    df = df.drop(drops, axis=1)
    df = df.rename(renames, axis=1)
    return df

clean_columns(df).head()
df.head()


,OBJECTID,UNITID,SIGID,MAINSTREET,CROSSSTREE,OWNER2,ROUTE,MILEPOINT,DESCRIPTIO,TIMES,OWNER,TYPE,INTID,ATMSID,geometry
0,1,MPW017206,SIG017206,OLD HENRY,I 265 RAMP,S,KY-3084,1.250,OLD HENRY @ I 265 RAMP,24 HOURS,KYTC,1,7674756999,NaN,POINT (-85.4993371456453 38.27063209239155)
1,2,MPW017211,SIG017211,SHELBYVILLE,BECKLEY WOODS,S,US 60,12.291,SHELBYVILLE @ BECKLEY WOODS,24 HOURS,KYTC,1,7895940672,NaN,POINT (-85.49847895741786 38.241589472827904)
2,3,MPW017290,SIG017290,SHELBYVILLE,LAKE FOREST,S,US-60,12.580,SHELBYVILLE @ LAKE FOREST,24 HOURS,KYTC,1,12375940763,NaN,POINT (-85.4935498603372 38.24051723243606)
3,4,MPW015159,SIG015159,BROWNSBORO,I 265 RAMP,S,KY-22,3.700,BROWNSBORO @ I 265 RAMP,24 HOURS,KYTC,1,14390724999,NaN,POINT (-85.57021228272409 38.31100207202721)
4,5,MPW011726,SIG011726,HERR,LIME KILN,S,KY-22,0.440,HERR @ LIME KILN,24 HOURS,KYTC,1,18422901386,NaN,POINT (-85.62796526747577 38.28391602421024)


### UNITID SIGID columns

In [124]:
assert df.UNITID.str.startswith("MPW").all()
assert df.SIGID.str.startswith("SIG").all()

mismatch = df[df.UNITID.str.lstrip("MPW") != df.SIGID.str.lstrip("SIG")]
mismatch.TYPE.value_counts() # == 27
# Dope. 

#df[(df.TYPE != 2) & (df.OWNER == 'KYTC')] 
# but not
#df[(df.TYPE == 2) & (df.OWNER != 'KYTC')] 

df[(df.TYPE != 2)].OWNER.value_counts()
df[df.OWNER == 'KYTC'].TYPE.value_counts()

# What does it mean?


TYPE
1    707
2     27
3     17
Name: count, dtype: int64

### ROUTES column

In [107]:
routes = df['ROUTE'].dropna().unique()
sorted(routes)

import pyparsing as pyp

prefix = pyp.Word(pyp.alphas)
sep = pyp.Literal('-')
#number = pyp.Word(pyp.nums)
suffix = pyp.Word(pyp.alphanums)


expr = prefix('road_prefix') + pyp.Optional(sep) +suffix('road_suffix')
expr.parse_string('KY-3084')
#expr.parse_string('KY 1934')
#expr.parse_string('KY-1447W')

parser = lambda s:expr.parse_string(s)

In [86]:
pd.DataFrame({route:parser(route).as_dict() for route in routes}).T

,road_prefix,road_suffix
KY-3084,KY,3084
US 60,US,60
US-60,US,60
KY-22,KY,22
US-42,US,42
KY-146,KY,146
KY-155,KY,155
US-31E,US,31E
KY-1747,KY,1747
KY-913,KY,913


#### TIMES column

In [87]:
df['TIMES'] = df['TIMES'].replace({"24 HOUR":"24 HOURS", "24HRS":"24 HOURS"})
df['TIMES'].value_counts()

TIMES
24 HOURS    1089
ACTUATED       1
Name: count, dtype: int64

In [88]:
df[df.TIMES == 'ACTUATED']
# TYPE == 4 # Hmm

,OBJECTID,UNITID,SIGID,MAINSTREET,CROSSSTREE,OWNER2,ROUTE,MILEPOINT,DESCRIPTIO,TIMES,OWNER,TYPE,INTID,ATMSID,geometry
1031,1032,MPW004166,SIG004166,BRECKINRIDGE,DIXIE,M,NaN,NaN,BRECKINRIDGE @ DIXIE,ACTUATED,METRO,4,131280658161,NaN,POINT (-85.78365846309039 38.24506461198751)


# TYPE column


In [102]:
df[['OWNER', 'TYPE']].value_counts()
# Nope. That didn't work
# ...but it DID work. Use up top: OWNER:KYTC, TYPE:2 <- count == 27

OWNER          TYPE
KYTC           1       707
METRO          1       305
KYTC           2        27
METRO          3        27
KYTC           3        17
JEFFERSONTOWN  1         5
METRO          4         2
MIDDLETOWN     1         1
Name: count, dtype: int64